# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [7]:
# imports
from IPython.display import display, update_display, Markdown
import os
from dotenv import load_dotenv
from openai import OpenAI 


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [18]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = """
You are python expert who is skilled in explaining code and helping people understand it.
Make sure to explain the code in a way that is easy to understand.
Keep your explanations concise and to the point.
Make sure to make it beginner friendly.
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content" : question}
]

In [19]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

stream = openai.chat.completions.create(model=MODEL_GPT, 
                                        messages= messages,
                                        stream=True)

gpt_response = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    gpt_response += chunk.choices[0].delta.content or ''
    update_display(Markdown(gpt_response), display_id=display_handle.display_id)


Certainly! Let's break down the code snippet you provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation:

1. **`{book.get("author") for book in books if book.get("author")}`**:
   - This part is a **set comprehension**. A set comprehension is used to create a set (a collection of unique items) in Python.
   - It iterates over a list called `books` (which is expected to be a collection of book data).
   - For each `book`, it tries to get the value associated with the key `"author"` using `book.get("author")`. 
   - If the author exists (meaning it’s not `None` or an empty string), it adds that author to the set.
   - The result is a set of unique author names from the list of books.

2. **`yield from`**:
   - This keyword is used in generator functions. It allows you to yield all values from an iterable (like a set or a list) one by one.
   - In this case, it means that for each unique author in the set that was created, a value will be returned each time the generator is iterated over. 

### Why It Matters:
- **Efficiency**: By using a set, it ensures that each author is unique and there are no duplicates.
- **Generator Function**: The use of `yield from` indicates that this code is likely part of a generator function, making it memory efficient since it generates values on-the-fly rather than storing them all in memory.

### Summary:
This code snippet efficiently gathers unique authors from a list of books and yields each author one at a time when the generator is iterated upon. It's a concise way to collect and return unique data from potentially large datasets without memory overhead.

In [20]:
# Get Llama 3.2 to answer
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

stream = ollama.chat.completions.create(model=MODEL_LLAMA, 
                                        messages= messages,
                                        stream=True)

ollama_response = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    ollama_response += chunk.choices[0].delta.content or ''
    update_display(Markdown(ollama_response), display_id=display_handle.display_id)

**Explanation:**

This code snippet is using a technique called "generator expression" to extract author names from a list of dictionaries. Let's break it down:

* `yield from`: This keyword is used to yield values from an iterable (in this case, a generator).
* `{book.get("author") for book in books if book.get("author")}`: This is a tuple-comprehension that performs the following operations:
	+ `for book in books`: Iterates over each dictionary (book) in the `books` list.
	+ `if book.get("author")`: Filters out dictionaries that don't have an author name. It uses the `.get()` method to safely retrieve a value from the dictionary, avoiding a KeyError if "author" doesn't exist.
	+ `book.get("author")`: Extracts the author's name (or None if it doesn't exist) from each filtered dictionary.
* `{...}`: This is an empty dictionary. It's used as a container to hold the extracted values.

**Why?**

The purpose of this code snippet is likely to provide a list of authors' names, where:

1. Not all books have an author name (in which case `None` will be included in the result).
2. The input data (`books`) might contain dictionaries with only certain keys (e.g., `title`, `description`, etc.), while this code assumes that each book dictionary must have a key named "author".

**Example use cases:**

When working with large datasets or data structures, generator expressions like this one can help you:

* Process data in chunks, avoiding excessive memory consumption
* Work with complex, nested data structures (in this case, a list of dictionaries)
* Improve code readability by encapsulating logic within a single expression

Keep in mind that the resulting values will be yielded on-the-fly as you iterate over them, rather than having all authors' names stored in memory at once.

In [21]:
# Compare the two responses using GPT-5-nano
comparison_prompt = f"""
You are an expert evaluator of educational content. Compare the following two explanations of the same Python code and rank them on a scale of 1-5 for each criterion:

**Criteria:**
1. **Beginner Friendliness**: How easy is it for a beginner to understand?
2. **Tackles Main Point**: How well does it address the core concept?
3. **Clear Examples**: How effective are the examples and explanations?

**Response A (GPT-4o-mini):**
{gpt_response}

**Response B (Llama 3.2):**
{ollama_response}

**Instructions:**
- Rate each response on each criterion from 1 (poor) to 5 (excellent)
- Provide a brief justification for each rating
- Give an overall recommendation on which response is better for a beginner

Please format your response clearly with the ratings and justifications.
"""

comparison_messages = [
    {"role": "user", "content": comparison_prompt}
]

stream = openai.chat.completions.create(
    model="gpt-5-nano",
    messages=comparison_messages,
    stream=True
)

comparison_response = ""
display_handle = display(Markdown("## 📊 Comparison Results\n\n"), display_id=True)

for chunk in stream:
    comparison_response += chunk.choices[0].delta.content or ''
    update_display(Markdown("## 📊 Comparison Results\n\n" + comparison_response), display_id=display_handle.display_id)


## 📊 Comparison Results

Here’s a side-by-side rating and justification for each criterion.

Response A (GPT-4o-mini)

- Beginner Friendliness: 4/5
  - Why: It explains the two main parts (set comprehension and yield from) in straightforward terms and uses bullet points. It’s easy to follow for someone with basic Python knowledge. Minor improvement could be more emphasis on what a generator function is and when you’d use this pattern.

- Tackles Main Point: 4/5
  - Why: It correctly identifies the core ideas: using a set comprehension to collect unique authors and using yield from to yield items from that iterable. It also notes potential memory-related considerations and the generator nature. The nuance about memory efficiency could be clarified (the set is built in memory first), but the core point is solid.

- Clear Examples: 4/5
  - Why: It provides a succinct, concrete breakdown of each piece of the expression and a concise summary of the result. It would be even better with a short concrete example of input and the yielded values, but the explanation is clear and actionable.

Response B (Llama 3.2)

- Beginner Friendliness: 2/5
  - Why: Contains several incorrect or misleading statements (calls the comprehension a “tuple-comprehension,” says the container is an empty dictionary, misdescribes yield from usage, and incorrectly claims that None would be yielded). These errors can confuse beginners and propagate misconceptions.

- Tackles Main Point: 2/5
  - Why: Several core misunderstandings (generator expression vs set comprehension, the effect of the if filter, and the nature of the resulting container) obscure the main concept. The explanation diverges from what the code does, reducing its usefulness as a core clarifier.

- Clear Examples: 2/5
  - Why: While it attempts to describe use cases and benefits of generator-like processing, the inaccuracies undermine the usefulness of the examples. It also incorrectly describes the resulting container and behavior of the code, making the examples less reliable for learning.

Overall recommendation

- Better for a beginner: Response A
  - Rationale: Response A is accurate, coherent, and focused on the essential ideas (set comprehension, deduplication, and how yield from works). It avoids the factual errors present in Response B and offers a clearer path to understanding the snippet. Response B contains multiple conceptual mistakes that could mislead beginners. If you want to improve Response B, you’d need to correct terminology (set vs. dict, generator vs. set comprehension) and fix the behavior description (no None is yielded due to the filter).